In [3]:
import pandas as pd 
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import math

In [4]:
import math
def next_power_of_2(x):
    """
    Find power of 2 greater than x
    """
    return 2 ** math.ceil(math.log(x) / math.log(2))


def low_pass(data, srate, fl):
    """
    low pass filter
    """
    oldlen = len(data)
    newlen = next_power_of_2(oldlen)

    # srate / nsamp = Frequency increment
    # (0 ~ nsamp-1) * srate / nsamp = frequency range
    y = np.fft.fft(data, newlen)

    # filtering
    half = math.ceil(newlen / 2)
    for i in range(half):
        f = i * srate / newlen
        if f > fl:
            y[i] = y[newlen - 1 - i] = 0

    # inverse transform
    return np.real(np.fft.ifft(y)[:oldlen])


def find_nearest(data, value):
    """
    Find the nearest value and return it
    :param data: array
    :param value: value to find
    :return: nearest value
    """
    idx = np.abs(np.array(data) - value).argmin()
    return data[idx]

def band_pass(data, srate, fl, fh):
    """
    band pass filter
    """
    if fl > fh:
        return band_pass(data, srate, fh, fl)

    oldlen = len(data)
    newlen = next_power_of_2(oldlen)

    # srate / nsamp = Frequency increment
    # (0 ~ nsamp-1) * srate / nsamp = frequency range
    y = np.fft.fft(data, newlen)

    # filtering
    half = math.ceil(newlen / 2)
    for i in range(half):
        f = i * srate / newlen
        if f < fl or f > fh:
            y[i] = y[newlen - 1 - i] = 0

    # inverse transform
    return np.real(np.fft.ifft(y)[:oldlen])



def max_idx(data, idxfrom = 0, idxto=None):
    idxfrom = max(0, idxfrom)
    if idxto == None:
        idxto = len(data)
    idxto = min(len(data), idxto)
    return idxfrom + np.argmax(data[idxfrom:idxto])

def detect_window_maxima(data, wind):
    """
    Returns the index of the sample which is the maximum from the samples in the specified window.
    """
    span = int(wind / 2)
    ret = []
    i = span
    while i < len(data) - span:  # Because i cannot be changed within a for statement
        idx_max = max_idx(data, i - span, i + span)
        if idx_max == i:
            ret.append(i)
        elif i < idx_max:
            i = idx_max - 1  # jump
        i += 1
    return ret


def detect_qrs(data, srate):
    """
    find qrs and return the indexes
    http://ocw.utm.my/file.php/38/SEB4223/07_ECG_Analysis_1_-_QRS_Detection.ppt%20%5BCompatibility%20Mode%5D.pdf
    """
    pcand = detect_window_maxima(data, 0.08 * srate)  # 80ms local peak

    y1 = band_pass(data, srate, 10, 20)  # The qrs value must be one of these.

    y2 = [0, 0]  # derivative
    for i in range(2, len(y1)-2):
        y2.append(2 * y1[i+2] + y1[i+1] - y1[i-1] - 2*y1[i-2])
    y2 += [0, 0]

    y3 = [x ** 2 for x in y2]  # Squaring

    # Moving Average filter
    # Acts as a smoother and performs a moving window integrator over 150ms
    y4 = []
    size = srate * 0.15
    for i in range(len(y3)):
        ifrom = max(0, i - int(size / 2))
        ito = min(i + int(size / 2), len(y3))
        y4.append(np.mean(y3[ifrom:ito]))

    y5 = band_pass(y4, srate, 0.5, 8)  # filtering -> remove drifting and abrupt change
    p1 = detect_window_maxima(y5, 0.3 * srate)  # find max in 300 ms

    # threshold -> 0.5 times the median value of the peak within 10 seconds before and after
    # peak_vals = []
    # for(i = 0 i < p1.length i++)
    #         peak_vals.push(y5[p1[i]])
    # th = peak_vals.median() * 0.5
    # peak_vals = null
    p2 = []
    for idx in p1:
        val = y5[idx]
        peak_vals = []
        for idx2 in p1:
            if abs(idx - idx2) < srate * 10:
                peak_vals.append(y5[idx])
        th = np.median(peak_vals) * 0.75
        if val >= th:
            p2.append(idx)

    # find closest peak
    p3 = []
    last = -1
    for x in p2:
        idx_cand = find_nearest(pcand, x)
        if idx_cand != last:
            p3.append(idx_cand)
        last = idx_cand

    # remove false positives (FP)
    i = 0
    while i < len(p3) - 1:
        idx1 = p3[i]
        idx2 = p3[i+1]
        if idx2 - idx1 < 0.2 * srate:  # physiological refractory period of about 200 ms
            if i == 0:
                dele = i
            elif i >= len(p3) - 2:
                dele = i + 1
            else:  # minimize heart rate variability
                idx_prev = p3[i-1]
                idx_next = p3[i+2]
                # find center point distance
                if abs(idx_next + idx_prev - 2 * idx1) > abs(idx_next + idx_prev - 2 * idx2):
                    dele = i
                else:
                    dele = i+1
            p3.pop(dele)
            if dele == i:
                i -= 1
        i += 1

    return p3

def remove_wander_spline(data, srate):
    """
    cubic spline ECG wander removal
    http://jh786t2saqs.tistory.com/416
    http://blog.daum.net/jty71/10850833
    """
    # calculate downslope
    downslope = [0, 0, 0]
    for i in range(3, len(data) - 3):
        downslope.append(data[i-3] + data[i-1] - data[i+1] - data[i+3])
    downslope += [0, 0, 0]

    r_list = detect_qrs(data, srate)  # detect r-peak

    rsize = int(0.060 * srate)  # knots from r-peak
    jsize = int(0.066 * srate)
    knots = []  # indexes of the kot
    for ridx in r_list:
        th = 0.6 * max(downslope[ridx:ridx + rsize])
        for j in range(ridx, ridx + rsize):
            if downslope[j] >= th:  # R detected
                knots.append(j - jsize)
                break

    # cubic spline for every knots
    baseline = [0] * len(data)
    for i in range(1, len(knots)-2):
        x1 = knots[i]
        x2 = knots[i+1]
        y1 = data[x1]
        y2 = data[x2]
        d1 = (data[x2] - data[knots[i-1]]) / (x2 - knots[i-1])
        d2 = (data[knots[i+2]] - data[x1]) / (knots[i+2] - x1)
        a = -2 * (y2-y1) / (x2-x1)**3 + (d2+d1) / (x2-x1)**2
        b = 3 * (y2-y1) / (x2-x1)**2 - (d2+2*d1) / (x2-x1)
        c = d1
        d = y1
        for x in range(x1, x2):
            x_a = (x-x1)  # x-a
            x_a2 = x_a * x_a
            x_a3 = x_a2 * x_a
            baseline[x] = a * x_a3 + b * x_a2 + c * x_a + d

    for i in range(len(data)):
        data[i] -= baseline[i]

    return data

In [7]:


root_folder = '../HDIdataset/'

case_id_list= []
event_mode_list = []
eeg_dataset = []
ecg_dataset = []
art_dataset = []

for file_name in os.listdir(root_folder):
    print(file_name)
    tmp = pd.read_csv(os.path.join(root_folder+file_name),index_col=0)
    case_id_list.append(file_name.split('.')[0])
    
    #### z-score normalization
    for col in tmp.columns:
        if col == 'EEG1_WAV':
            a = tmp[col].dropna()

            #### z-score normalization
            tmp_mean = a.mean()
            tmp_std = a.std()
            a = (a - tmp_mean) / (tmp_std+1e-5)
            eeg_dataset.append (list([v for v in a ]))


        elif col == 'ART':
            a = tmp[col].dropna()

        #### z-score normalization
            tmp_mean = a.mean()
            tmp_std = a.std()
            a = (a - tmp_mean) / (tmp_std+1e-5)
            art_dataset.append (list([v for v in a ]))

        elif col == 'ECG_II':
            a = tmp[col].dropna()
            
            if(len(a)>0):
                try:
                    a= remove_wander_spline(np.array(a),250)
                except ValueError:
                    pass
            #### z-score normalization
            tmp_mean = a.mean()
            tmp_std = a.std()
            a = (a - tmp_mean) / (tmp_std+1e-5)
            ecg_dataset.append (list([v for v in a ]))
        
    
    if file_name.split('.')[1] == 'e':
        event_mode_list.append(1)
    else:
        event_mode_list.append(0)
    

eeg_dataset_pd= pd.DataFrame(eeg_dataset)
eeg_dataset_pd['CASEID'] = pd.DataFrame(case_id_list)
eeg_dataset_pd['EVENT'] = pd.DataFrame(event_mode_list)

ecg_dataset_pd= pd.DataFrame(ecg_dataset)
ecg_dataset_pd['CASEID'] = pd.DataFrame(case_id_list)
ecg_dataset_pd['EVENT'] = pd.DataFrame(event_mode_list)

art_dataset_pd= pd.DataFrame(art_dataset)
art_dataset_pd['CASEID'] = pd.DataFrame(case_id_list)
art_dataset_pd['EVENT'] = pd.DataFrame(event_mode_list)
#print(pd.read_csv(os.path.join(root_folder+file_path),index_col=0).head())
     
    


00060.n.003.csv
00041.n.002.csv
00017.n.000.csv
00029.n.005.csv
00113.n.016.csv
00113.n.002.csv
00055.n.002.csv
00101.n.005.csv
00079.n.000.csv
00074.n.017.csv
00074.n.003.csv
00016.e.csv
00074.n.002.csv
00074.n.016.csv
00079.n.001.csv
00101.n.004.csv
00055.n.003.csv
00113.n.003.csv
00113.n.017.csv
00029.n.004.csv
00017.n.001.csv
00060.n.002.csv
00097.n.005.csv
00060.n.000.csv
00005.n.004.csv
00029.e.csv
00041.n.001.csv
00029.n.006.csv
00113.n.001.csv
00113.n.015.csv
00055.n.001.csv
00113.n.029.csv
00079.n.003.csv
00074.n.000.csv
00074.n.014.csv
00075.e.csv
00074.n.015.csv
00074.n.001.csv
00079.n.002.csv
00113.n.028.csv
00055.n.000.csv
00113.n.014.csv
00113.n.000.csv
00017.n.002.csv
00029.n.007.csv
00041.n.000.csv
00005.n.005.csv
00060.n.001.csv
00097.n.004.csv
00027.n.008.csv
00097.n.000.csv
00060.n.005.csv
00005.n.001.csv
00029.n.003.csv
00055.n.004.csv
00113.n.004.csv
00113.n.010.csv
00066.n.002.csv
.DS_Store
00101.n.003.csv
00114.e.csv
00074.n.005.csv
00074.n.011.csv
00083.n.000.cs

In [15]:
# split ids
from sklearn.model_selection import train_test_split
id_fullset = art_dataset_pd['CASEID'].copy()
training_ids, test_ids = train_test_split \
                                    (id_fullset,  test_size=0.3, random_state=42)

In [16]:
#make training set
eeg_dataset_pd[eeg_dataset_pd['CASEID'].isin(training_ids)].to_csv('eeg_training.csv')
ecg_dataset_pd[ecg_dataset_pd['CASEID'].isin(training_ids)].to_csv('ecg_training.csv')
art_dataset_pd[art_dataset_pd['CASEID'].isin(training_ids)].to_csv('art_training.csv')

In [17]:
#make test set
eeg_dataset_pd[eeg_dataset_pd['CASEID'].isin(test_ids)].to_csv('eeg_test.csv')
ecg_dataset_pd[ecg_dataset_pd['CASEID'].isin(test_ids)].to_csv('ecg_test.csv')
art_dataset_pd[art_dataset_pd['CASEID'].isin(test_ids)].to_csv('art_test.csv')